<a href="https://colab.research.google.com/github/Geriys/VKirillV/blob/main/HW_3_ML2-ok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, make_scorer, r2_score
from sklearn.feature_selection import SelectKBest
from sklearn.datasets import fetch_openml, make_regression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from scipy.stats import randint


In [33]:
RANDOM_STATE = 42

results_regression = pd.DataFrame(columns = ['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns = ['model', 'task', 'f1', 'accuracy'])

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [34]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [45]:

### Ваш код ###
from sklearn.model_selection import train_test_split
y = data["MEDV"]
X = data.drop(columns=["MEDV"])
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 10, shuffle = True)

2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [36]:

### Ваш код ###
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
# lr_y_pred = lr_model.predict(X_test)

ridge_model = Ridge()
ridge_model.fit(X_train, y_train)
# ridge_y_pred = ridge_model.predict(X_test)

lasso_model = Lasso()
lasso_model.fit(X_train, y_train)
# lasso_y_pred = lasso_model.predict(X_test)

# https://scikit-learn.ru/3-1-cross-validation-evaluating-estimator-performance/
# https://datafinder.ru/products/ocenka-r2-v-mashinnom-obuchenii#:~:text=%D0%9E%D1%86%D0%B5%D0%BD%D0%BA%D0%B0%20R2%20%E2%80%93%20%D0%BE%D1%87%D0%B5%D0%BD%D1%8C%20%D0%B2%D0%B0%D0%B6%D0%BD%D1%8B%D0%B9%20%D0%BF%D0%BE%D0%BA%D0%B0%D0%B7%D0%B0%D1%82%D0%B5%D0%BB%D1%8C,%D0%B2%20%D0%BF%D1%80%D0%BE%D0%B3%D0%BD%D0%BE%D0%B7%D0%B0%D1%85%2C%20%D0%BE%D0%B1%D1%8A%D1%8F%D1%81%D0%BD%D0%B5%D0%BD%D0%BD%D1%8B%D1%85%20%D0%BD%D0%B0%D0%B1%D0%BE%D1%80%D0%BE%D0%BC%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.
# Calculation of R2 Score
r2_lr = lr_model.score(X_test, y_test) #cross_val_score(lr_model, X_test, y_test, cv=KFold(n_splits=3), scoring="r2").mean()
r2_ridge = ridge_model.score(X_test, y_test) #cross_val_score(ridge_model, X_test, y_test, cv=KFold(n_splits=3), scoring="r2").mean()
r2_lasso = lasso_model.score(X_test, y_test) #cross_val_score(lasso_model, X_test, y_test, cv=KFold(n_splits=3), scoring="r2").mean()

results_regression.loc[0] = ['LR', 'task2', r2_lr]
results_regression.loc[1] = ['Ridge', 'task2', r2_ridge]
results_regression.loc[2] = ['Lasso', 'task2', r2_lasso]
print(results_regression)

   model   task        R2
0     LR  task2  0.670642
1  Ridge  task2  0.665291
2  Lasso  task2  0.534676


3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [37]:

# https://medium.com/nuances-of-programming/%D0%BB%D0%B0%D1%81%D1%81%D0%BE-%D0%B8-%D1%80%D0%B8%D0%B4%D0%B6-%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D0%B8-%D0%B8%D0%BD%D1%82%D1%83%D0%B8%D1%82%D0%B8%D0%B2%D0%BD%D0%BE%D0%B5-%D1%81%D1%80%D0%B0%D0%B2%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5-a542dd761f62
### Ваш код ###


# Defining grid of candidate alpha values (powers of 10, from 0.00001 to 1000000)
param_grid = {"alpha": 10.0 ** np.arange(-5, 6)}

# Initializing Ridge and GridSearchCV estimators
ridge = Ridge()
print(ridge.get_params().keys())
grid_search = GridSearchCV(estimator=ridge, param_grid=param_grid)
# Fitting grid search object
grid_search.fit(X_train, y_train)
r2_ridge_grid_search = grid_search.score(X_test, y_test)



ridge_reg_cv = RidgeCV(alphas=param_grid['alpha'], cv=5)
ridge_reg_cv.fit(X_train, y_train)
r2_ridge_cv = ridge_reg_cv.score(X_test, y_test)



# Initializing Lasso and GridSearchCV estimators
lasso = Lasso()
grid_search_lasso = GridSearchCV(estimator=lasso, param_grid=param_grid)
# Fitting grid search object
grid_search_lasso.fit(X_train, y_train)
r2_lasso_grid_search = grid_search_lasso.score(X_test, y_test)



lasso_reg_cv = LassoCV(alphas=param_grid['alpha'], cv=5)
lasso_reg_cv.fit(X_train, y_train)

r2_lasso_cv = lasso_reg_cv.score(X_test, y_test)


results_regression.loc[3] = ['Ridge_GridSearchCV', 'task3', r2_ridge_grid_search]
results_regression.loc[4] = ['RidgeCV', 'task3', r2_ridge_cv]
results_regression.loc[5] = ['Lasso_GridSearchCV', 'task3', r2_lasso_grid_search]
results_regression.loc[6] = ['LassoCV', 'task3', r2_lasso_cv]
print(results_regression)

dict_keys(['alpha', 'copy_X', 'fit_intercept', 'max_iter', 'positive', 'random_state', 'solver', 'tol'])
                model   task        R2
0                  LR  task2  0.670642
1               Ridge  task2  0.665291
2               Lasso  task2  0.534676
3  Ridge_GridSearchCV  task3  0.670002
4             RidgeCV  task3  0.670002
5  Lasso_GridSearchCV  task3  0.670360
6             LassoCV  task3  0.670360


4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [38]:
### Ваш код ###

estimators = [('scaler', StandardScaler()), ('model', Ridge())]
pipe = Pipeline(estimators)
r2_ridge_standart_scaler = pipe.fit(X_train, y_train).score(X_test, y_test)

estimators = [('scaler', MinMaxScaler()), ('model', Ridge())]
pipe = Pipeline(estimators)
r2_ridge_min_max_scaler = pipe.fit(X_train, y_train).score(X_test, y_test)

estimators = [('scaler', StandardScaler()), ('model', Lasso())]
pipe = Pipeline(estimators)
r2_lasso_standart_scaler = pipe.fit(X_train, y_train).score(X_test, y_test)

estimators = [('scaler', MinMaxScaler()), ('model', Lasso())]
pipe = Pipeline(estimators)
r2_lasso_min_max_scaler = pipe.fit(X_train, y_train).score(X_test, y_test)
results_regression.loc[7] = ['Ridge_StandardScaler', 'task4', r2_ridge_standart_scaler]
results_regression.loc[8] = ['Ridge_MinMaxScaler', 'task4', r2_ridge_min_max_scaler]
results_regression.loc[9] = ['Lasso_StandardScaler', 'task4', r2_lasso_standart_scaler]
results_regression.loc[10] = ['Lasso_MinMaxScaler', 'task4', r2_lasso_min_max_scaler]
print(results_regression)

                   model   task        R2
0                     LR  task2  0.670642
1                  Ridge  task2  0.665291
2                  Lasso  task2  0.534676
3     Ridge_GridSearchCV  task3  0.670002
4                RidgeCV  task3  0.670002
5     Lasso_GridSearchCV  task3  0.670360
6                LassoCV  task3  0.670360
7   Ridge_StandardScaler  task4  0.670844
8     Ridge_MinMaxScaler  task4  0.666259
9   Lasso_StandardScaler  task4  0.608150
10    Lasso_MinMaxScaler  task4  0.094319


5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [39]:
### Ваш код ###

pipe = Pipeline([('scaler', StandardScaler()), ('model', Ridge())])
params = { 'model__alpha': 10.0 ** np.arange(-5, 6) }
grid_pipeline = GridSearchCV(pipe, params)
r2_ridge_standart_scaler_cv = grid_pipeline.fit(X_train, y_train).score(X_test, y_test)

pipe = Pipeline([('scaler', MinMaxScaler()), ('model', Ridge())])
params = { 'model__alpha': 10.0 ** np.arange(-5, 6) }
grid_pipeline = GridSearchCV(pipe, params)
r2_ridge_min_max_scaler_cv = grid_pipeline.fit(X_train, y_train).score(X_test, y_test)

pipe = Pipeline([('scaler', StandardScaler()), ('model', Lasso())])
params = { 'model__alpha': 10.0 ** np.arange(-5, 6) }
grid_pipeline = GridSearchCV(pipe, params)
r2_lasso_standart_scaler_cv = grid_pipeline.fit(X_train, y_train).score(X_test, y_test)

pipe = Pipeline([('scaler', MinMaxScaler()), ('model', Lasso())])
params = { 'model__alpha': 10.0 ** np.arange(-5, 6) }
grid_pipeline = GridSearchCV(pipe, params)
r2_lasso_min_max_scaler_cv = grid_pipeline.fit(X_train, y_train).score(X_test, y_test)

results_regression.loc[11] = ['Ridge_StandardScaler_CV', 'task5', r2_ridge_standart_scaler_cv]
results_regression.loc[12] = ['Ridge_MinMaxScaler_CV', 'task5', r2_ridge_min_max_scaler_cv]
results_regression.loc[13] = ['Lasso_StandardScaler_CV', 'task5', r2_lasso_standart_scaler_cv]
results_regression.loc[14] = ['Lasso_MinMaxScaler_CV', 'task5', r2_lasso_min_max_scaler_cv]
print(results_regression)

                      model   task        R2
0                        LR  task2  0.670642
1                     Ridge  task2  0.665291
2                     Lasso  task2  0.534676
3        Ridge_GridSearchCV  task3  0.670002
4                   RidgeCV  task3  0.670002
5        Lasso_GridSearchCV  task3  0.670360
6                   LassoCV  task3  0.670360
7      Ridge_StandardScaler  task4  0.670844
8        Ridge_MinMaxScaler  task4  0.666259
9      Lasso_StandardScaler  task4  0.608150
10       Lasso_MinMaxScaler  task4  0.094319
11  Ridge_StandardScaler_CV  task5  0.671640
12    Ridge_MinMaxScaler_CV  task5  0.670629
13  Lasso_StandardScaler_CV  task5  0.671214
14    Lasso_MinMaxScaler_CV  task5  0.671900


6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [40]:
### Ваш код ###

pipe = Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures()), ('model', Ridge())])
r2_ridge_standart_scaler_poly = pipe.fit(X_train, y_train).score(X_test, y_test)

pipe = Pipeline([('scaler', MinMaxScaler()), ('poly', PolynomialFeatures()), ('model', Ridge())])
r2_ridge_min_max_scaler_poly = pipe.fit(X_train, y_train).score(X_test, y_test)

pipe = Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures()), ('model', Lasso())])
r2_lasso_standart_scaler_poly = pipe.fit(X_train, y_train).score(X_test, y_test)

pipe = Pipeline([('scaler', MinMaxScaler()), ('poly', PolynomialFeatures()), ('model', Lasso())])
r2_lasso_min_max_scaler_poly = pipe.fit(X_train, y_train).score(X_test, y_test)


results_regression.loc[15] = ['Ridge_StandardScaler_Poly', 'task6', r2_ridge_standart_scaler_poly]
results_regression.loc[16] = ['Ridge_MinMaxScaler_Poly', 'task6', r2_ridge_min_max_scaler_poly]
results_regression.loc[17] = ['Lasso_StandardScaler_Poly', 'task6', r2_lasso_standart_scaler_poly]
results_regression.loc[18] = ['Lasso_MinMaxScaler_Poly', 'task6', r2_lasso_min_max_scaler_poly]
print(results_regression)

                        model   task        R2
0                          LR  task2  0.670642
1                       Ridge  task2  0.665291
2                       Lasso  task2  0.534676
3          Ridge_GridSearchCV  task3  0.670002
4                     RidgeCV  task3  0.670002
5          Lasso_GridSearchCV  task3  0.670360
6                     LassoCV  task3  0.670360
7        Ridge_StandardScaler  task4  0.670844
8          Ridge_MinMaxScaler  task4  0.666259
9        Lasso_StandardScaler  task4  0.608150
10         Lasso_MinMaxScaler  task4  0.094319
11    Ridge_StandardScaler_CV  task5  0.671640
12      Ridge_MinMaxScaler_CV  task5  0.670629
13    Lasso_StandardScaler_CV  task5  0.671214
14      Lasso_MinMaxScaler_CV  task5  0.671900
15  Ridge_StandardScaler_Poly  task6  0.877763
16    Ridge_MinMaxScaler_Poly  task6  0.825928
17  Lasso_StandardScaler_Poly  task6  0.748369
18    Lasso_MinMaxScaler_Poly  task6  0.089644


7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [41]:
### Ваш код ###

params = { 'model__alpha': 10.0 ** np.arange(-5, 6) }

pipe = Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures()), ('model', Ridge())])
grid_pipeline = GridSearchCV(pipe, params)
r2_ridge_standart_scaler_poly_cv = grid_pipeline.fit(X_train, y_train).score(X_test, y_test)

pipe = Pipeline([('scaler', MinMaxScaler()), ('poly', PolynomialFeatures()), ('model', Ridge())])
grid_pipeline = GridSearchCV(pipe, params)
r2_ridge_min_max_scaler_poly_cv = grid_pipeline.fit(X_train, y_train).score(X_test, y_test)

pipe = Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures()), ('model', Lasso())])
grid_pipeline = GridSearchCV(pipe, params)
r2_lasso_standart_scaler_poly_cv = grid_pipeline.fit(X_train, y_train).score(X_test, y_test)

pipe = Pipeline([('scaler', MinMaxScaler()), ('poly', PolynomialFeatures()), ('model', Lasso())])
grid_pipeline = GridSearchCV(pipe, params)
r2_lasso_min_max_scaler_poly_cv = grid_pipeline.fit(X_train, y_train).score(X_test, y_test)


results_regression.loc[19] = ['Ridge_StandardScaler_Poly_CV', 'task7', r2_ridge_standart_scaler_poly_cv]
results_regression.loc[20] = ['Ridge_MinMaxScaler_Poly_CV', 'task7', r2_ridge_min_max_scaler_poly_cv]
results_regression.loc[21] = ['Lasso_StandardScaler_Poly_CV', 'task7', r2_lasso_standart_scaler_poly_cv]
results_regression.loc[22] = ['Lasso_MinMaxScaler_Poly_CV', 'task7', r2_lasso_min_max_scaler_poly_cv]
print(results_regression)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.495e+02, tolerance: 2.514e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.146e+02, tolerance: 2.574e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.237e+02, tolerance: 2.353e

                           model   task        R2
0                             LR  task2  0.670642
1                          Ridge  task2  0.665291
2                          Lasso  task2  0.534676
3             Ridge_GridSearchCV  task3  0.670002
4                        RidgeCV  task3  0.670002
5             Lasso_GridSearchCV  task3  0.670360
6                        LassoCV  task3  0.670360
7           Ridge_StandardScaler  task4  0.670844
8             Ridge_MinMaxScaler  task4  0.666259
9           Lasso_StandardScaler  task4  0.608150
10            Lasso_MinMaxScaler  task4  0.094319
11       Ridge_StandardScaler_CV  task5  0.671640
12         Ridge_MinMaxScaler_CV  task5  0.670629
13       Lasso_StandardScaler_CV  task5  0.671214
14         Lasso_MinMaxScaler_CV  task5  0.671900
15     Ridge_StandardScaler_Poly  task6  0.877763
16       Ridge_MinMaxScaler_Poly  task6  0.825928
17     Lasso_StandardScaler_Poly  task6  0.748369
18       Lasso_MinMaxScaler_Poly  task6  0.089644


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.853e+01, tolerance: 3.109e+00
  model = cd_fast.enet_coordinate_descent(


8. Подберите наилучшую модель (используйте Pipeline, GridSearchCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [48]:

# Создаем список моделей с различными параметрами
models = [
    ('Ridge', Ridge(), {
        'regressor__alpha': [0.1, 1, 10],
        'scaler': [StandardScaler(), 'passthrough'],
        'poly__degree': [2, 3],
    }),
    ('Lasso', Lasso(max_iter=10000), {
        'regressor__alpha': [0.1, 1, 10],
        'scaler': [StandardScaler(), 'passthrough'],
        'poly__degree': [2, 3],
    }),
]

# Проходимся по моделям и настраиваем параметры с помощью GridSearchCV
r2_best_model = 0
best_params = {}
best_model_name = ""

for model_name, model, params in models:
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('poly', PolynomialFeatures()),
        ('regressor', model)
    ])

    grid_search = GridSearchCV(pipe, params, cv=5, scoring='r2', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Оцениваем модель на тестовых данных
    y_pred = grid_search.predict(X_test)
    score = r2_score(y_test, y_pred)

    # Проверяем, является ли это лучшей моделью
    if score > r2_best_model:
        r2_best_model = score
        best_params = grid_search.best_params_
        best_model_name = model_name

# Выводим лучшие параметры и результат R2
print(f"Лучшая модель: {best_model_name}")
print(f"Лучшие параметры: {best_params}")
print(f"Лучший результат R2: {r2_best_model:.4f}")

# print('Параметры лучшей модели:\n', best_params)
# r2_best_model = 0
results_regression.loc[23] = ['Best_Model', 'task8', r2_best_model]

Лучшая модель: Ridge
Лучшие параметры: {'poly__degree': 2, 'regressor__alpha': 10, 'scaler': StandardScaler()}
Лучший результат R2: 0.8666


In [49]:
results_regression

,model,task,R2
0,LR,task2,0.670642
1,Ridge,task2,0.665291
2,Lasso,task2,0.534676
3,Ridge_GridSearchCV,task3,0.670002
4,RidgeCV,task3,0.670002
5,Lasso_GridSearchCV,task3,0.670360
6,LassoCV,task3,0.670360
7,Ridge_StandardScaler,task4,0.670844
8,Ridge_MinMaxScaler,task4,0.666259
9,Lasso_StandardScaler,task4,0.608150


http://archive.ics.uci.edu/ml/datasets/Adult

In [18]:
data = pd.read_csv('adult.csv')
print(data['class'].unique())#.head()

['<=50K' '>50K']


9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [19]:

### Ваш код ###
data["class-num"] = np.where(data["class"] == "<=50K", 1, 0)
y = data["class-num"]
X = data.drop(columns=["class", "class-num"])
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 10, shuffle = True)


10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [20]:
#Определить самый часто встречающийся класс 1 или 0.
#Обучить модель фитом как обычно // треин тест сплит на отфильтрованой выборке самого частого класса
#на том что получилось найти аккураси и ф1 скор

### Ваш код ###
print(len(data[data["class-num"] == 1]))
print(len(data[data["class-num"] == 0]))
# Самый частый класс - 1
# most_frequent_class = data[data["class-num"] == 1] # сделали датасет только с самым частым классом
# y_most_frequent = most_frequent_class["class-num"] # целевая переменная
# X_most_frequent = most_frequent_class.drop(columns=["class", "class-num"]) # признаки с самым частым классом

# params = { 'model__alpha': 10.0 ** np.arange(-5, 6) }

# pipe = Pipeline([('scaler', MinMaxScaler()), ('poly', PolynomialFeatures()), ('model', Ridge())])
# grid_pipeline = GridSearchCV(pipe, params)
# grid_pipeline.fit(X_train, y_train) # обучение модели
# y_pred_mf = grid_pipelinve.predict(X_most_frequent) # предсказание самого частого класса




# X_train_most_frequent = X_test[X_test["class-num"] == 1]
# y_train_most_frequent = y_test[y_test["class-num"] == 1]

# print([1] * len(y_train))

f1_most_frequent = f1_score(y_train, [1] * len(y_train))
acc_most_frequent = accuracy_score(y_train, [1] * len(y_train))
results_classification.loc[0] = ['Most Frequent class', 'task10', f1_most_frequent, acc_most_frequent]
print(results_classification)

37155
11687
                 model    task        f1  accuracy
0  Most Frequent class  task10  0.864899  0.761958


11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [21]:
# columns_with_nulls = ["workclass", "occupation", "native-country"]
# data[data["native-country"] == '?']

imp = SimpleImputer(missing_values='?', strategy="most_frequent", copy=False)
X["workclass"] = imp.fit_transform(X["workclass"].values.reshape(-1,1))[:,0]
X["occupation"] = imp.fit_transform(X["occupation"].values.reshape(-1,1))[:,0]
X["native-country"] = imp.fit_transform(X["native-country"].values.reshape(-1,1))[:,0]

X[X["native-country"] == '?']
# data.dropna()
### Ваш код ###


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country


12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [22]:

### Ваш код ###
data.info()
numeric_cols = X_train.columns[X_train.dtypes == 'int64'].tolist()
category_cols = X_train.columns[X_train.dtypes == 'object'].tolist()

print(f'num columns: {numeric_cols}')
print(f'category_columns: {category_cols}')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       48842 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      48842 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48842 non-null  object
 14  class           48842 non-null  object
 15  class-num       48842 non-null  int64 
dtypes: int64(7), object(9)
memory usage: 6.0+ MB
num columns: ['age', 'fnlwgt', 'education-num', '

13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [23]:

### Ваш код ###
# https://stackoverflow.com/questions/71715754/valueerror-specifying-the-columns-using-strings-is-only-supported-for-pandas-da
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', MinMaxScaler(), numeric_cols),
        ('categorical', OneHotEncoder(), category_cols)
    ], remainder='passthrough'
)


X_output = preprocessor.fit_transform(X_train)
print(X_output)
logistic_regression = LogisticRegression()
k_neighbors_classifier = KNeighborsClassifier()
linear_svc = LinearSVC()

pipeline_lr = Pipeline(steps=[('col_tra', ColumnTransformer(remainder='passthrough',
                                             transformers=[('trf', MinMaxScaler(), numeric_cols),
                                                           ('categorical', OneHotEncoder(), category_cols)])),
               ('model', logistic_regression)])


# pipeline_lr = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', logistic_regression)])

pipeline_lr = make_pipeline(preprocessor, logistic_regression)
pipeline_knn = make_pipeline(preprocessor, k_neighbors_classifier)
pipeline_svm = make_pipeline(preprocessor, linear_svc)

# pipeline_lr.fit(X_output, y_train)
# pipeline_knn.fit(X_output, y_train)
# pipeline_svm.fit(X_output, y_train)

# scores_lr = cross_val_score(pipeline_lr, X_test, y_test, cv=5, scoring=['accuracy', 'f1'])
# scores_knn = cross_val_score(pipeline_knn, X_test, y_test, cv=5, scoring=['accuracy', 'f1'])
# scores_svm = cross_val_score(pipeline_svm, X_test, y_test, cv=5, scoring=['accuracy', 'f1'])

# print(scores_lr)

f1_LR = cross_val_score(pipeline_lr, X_test, y_test, cv=5, scoring='f1')
acc_LR = cross_val_score(pipeline_lr, X_test, y_test, cv=5, scoring='accuracy')
f1_KNN = cross_val_score(pipeline_knn, X_test, y_test, cv=5, scoring='f1')
acc_KNN = cross_val_score(pipeline_knn, X_test, y_test, cv=5, scoring='accuracy')
f1_SVM = cross_val_score(pipeline_svm, X_test, y_test, cv=5, scoring='f1')
acc_SVM = cross_val_score(pipeline_svm, X_test, y_test, cv=5, scoring='accuracy')

results_classification.loc[1] = ['LogisticRegression', 'task13', f1_LR, acc_LR]
results_classification.loc[2] = ['KNeighborsClassifier', 'task13', f1_KNN, acc_KNN]
results_classification.loc[3] = ['LinearSVC', 'task13', f1_SVM, acc_SVM]
print(results_classification)

  (0, 0)	0.3835616438356165
  (0, 1)	0.1491677206704818
  (0, 2)	0.06666666666666667
  (0, 5)	0.39795918367346933
  (0, 10)	1.0
  (0, 18)	1.0
  (0, 33)	1.0
  (0, 45)	1.0
  (0, 53)	1.0
  (0, 63)	1.0
  (0, 65)	1.0
  (0, 92)	1.0
  (1, 0)	0.3698630136986301
  (1, 1)	0.13219103695016884
  (1, 2)	0.5333333333333333
  (1, 5)	0.39795918367346933
  (1, 6)	1.0
  (1, 26)	1.0
  (1, 33)	1.0
  (1, 38)	1.0
  (1, 58)	1.0
  (1, 63)	1.0
  (1, 64)	1.0
  (1, 105)	1.0
  (2, 0)	0.4246575342465753
  :	:
  (39070, 105)	1.0
  (39071, 0)	0.2191780821917808
  (39071, 1)	0.0086396715299802
  (39071, 2)	0.7333333333333334
  (39071, 5)	0.39795918367346933
  (39071, 10)	1.0
  (39071, 22)	1.0
  (39071, 35)	1.0
  (39071, 46)	1.0
  (39071, 57)	1.0
  (39071, 59)	1.0
  (39071, 64)	1.0
  (39071, 105)	1.0
  (39072, 0)	0.0273972602739726
  (39072, 1)	0.0567191727582219
  (39072, 2)	0.5333333333333333
  (39072, 5)	0.39795918367346933
  (39072, 10)	1.0
  (39072, 26)	1.0
  (39072, 35)	1.0
  (39072, 43)	1.0
  (39072, 54)	1.0
  

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 276

                  model    task  \
0   Most Frequent class  task10   
1    LogisticRegression  task13   
2  KNeighborsClassifier  task13   
3             LinearSVC  task13   

                                                  f1  \
0                                           0.864899   
1  [nan, 0.8978653530377668, 0.8955901426718547, ...   
2  [nan, 0.8794373744139318, 0.8829300196206671, ...   
3  [nan, 0.9031413612565444, 0.9032467532467534, ...   

                                            accuracy  
0                                           0.761958  
1  [nan, 0.8408393039918116, 0.8352098259979529, ...  
2  [nan, 0.8157625383828045, 0.8167860798362334, ...  
3  [nan, 0.8485158648925282, 0.8474923234390993, ...  


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 276, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 480, in predict
    Xt = transform.transform(Xt)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_set_output.py", line 140, in wrapped
    data_to_wrap = f(self, X, *args, **kwargs)
  File "/usr/local/

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [24]:
# Определение числовых и категориальных колонок
numeric_columns = X.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = X.select_dtypes(include=['object']).columns

# Создание преобразователей для числовых и категориальных колонок
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Создание колонного преобразователя
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# Создание пайплайна
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Преобразование данных
X_transformed = pipeline.fit_transform(X)

# Алгоритмы классификации
classifiers = [
    ('Logistic Regression', LogisticRegression(max_iter=1000)),
    ('K-Nearest Neighbors', KNeighborsClassifier()),
    ('Linear SVC', LinearSVC(max_iter=1000))
]

# Метрики
metrics = {
    'accuracy': make_scorer(accuracy_score),
    'f1_score': make_scorer(f1_score)
}

# Список для хранения результатов
results = []

# Вычисление cross_val_score для каждого алгоритма и каждой метрики
for name, clf in classifiers:
    print(f"Classifier: {name}")
    for metric_name, metric in metrics.items():
        scores = cross_val_score(clf, X_transformed, y, cv=5, scoring=metric)
        mean_score = scores.mean()
        std_dev = scores.std() * 2
        print(f"{metric_name}: {mean_score:.4f} (+/- {std_dev:.4f})")
        # Добавление результатов в список
        results.append((name, metric_name, mean_score, std_dev))
    print()

# Запись результатов в переменные
for result in results:
    if result[0] == 'Logistic Regression':
        if result[1] == 'accuracy':
            acc_LR = result[2]
        elif result[1] == 'f1_score':
            f1_LR = result[2]
    elif result[0] == 'K-Nearest Neighbors':
        if result[1] == 'accuracy':
            acc_KNN = result[2]
        elif result[1] == 'f1_score':
            f1_KNN = result[2]
    elif result[0] == 'Linear SVC':
        if result[1] == 'accuracy':
            acc_SVM = result[2]
        elif result[1] == 'f1_score':
            f1_SVM = result[2]

# Вывод переменных с результатами
print(f"f1_LR: {f1_LR}")
print(f"acc_LR: {acc_LR}")
print(f"f1_KNN: {f1_KNN}")
print(f"acc_KNN: {acc_KNN}")
print(f"f1_SVM: {f1_SVM}")
print(f"acc_SVM: {acc_SVM}")


results_classification.loc[4] = ['LogisticRegression_impute', 'task14', f1_LR, acc_LR]
results_classification.loc[5] = ['KNeighborsClassifier_impute', 'task14', f1_KNN, acc_KNN]
results_classification.loc[6] = ['LinearSVC_impute', 'task14', f1_SVM, acc_SVM]
results_classification

Classifier: Logistic Regression
accuracy: 0.8505 (+/- 0.0058)
f1_score: 0.9046 (+/- 0.0038)

Classifier: K-Nearest Neighbors
accuracy: 0.8249 (+/- 0.0030)
f1_score: 0.8872 (+/- 0.0019)

Classifier: Linear SVC
accuracy: 0.8508 (+/- 0.0035)
f1_score: 0.9051 (+/- 0.0023)

f1_LR: 0.9046421273044345
acc_LR: 0.8504566501751475
f1_KNN: 0.8871526909906728
acc_KNN: 0.824945711538505
f1_SVM: 0.905141981677368
acc_SVM: 0.8508251523375897


,model,task,f1,accuracy
0,Most Frequent class,task10,0.864899,0.761958
1,LogisticRegression,task13,"[nan, 0.8978653530377668, 0.8955901426718547, ...","[nan, 0.8408393039918116, 0.8352098259979529, ..."
2,KNeighborsClassifier,task13,"[nan, 0.8794373744139318, 0.8829300196206671, ...","[nan, 0.8157625383828045, 0.8167860798362334, ..."
3,LinearSVC,task13,"[nan, 0.9031413612565444, 0.9032467532467534, ...","[nan, 0.8485158648925282, 0.8474923234390993, ..."
4,LogisticRegression_impute,task14,0.904642,0.850457
5,KNeighborsClassifier_impute,task14,0.887153,0.824946
6,LinearSVC_impute,task14,0.905142,0.850825


15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [25]:


# Загрузка данных и удаление строк с '?'
# data = pd.read_csv('adult.csv')
# X = data.drop(columns=["class", "class-num"])
# y = data["class-num"]

# Удаление строк с '?'
X = X.replace('?', np.nan).dropna()
y = y[X.index]

# Определение числовых и категориальных колонок
numeric_columns = X.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = X.select_dtypes(include=['object']).columns

# Создание преобразователей для числовых и категориальных колонок
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Создание колонного преобразователя
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# Создание пайплайна
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Преобразование данных
X_transformed = pipeline.fit_transform(X)

# Алгоритмы классификации
classifiers = [
    ('Logistic Regression', LogisticRegression(max_iter=100)),
    ('K-Nearest Neighbors', KNeighborsClassifier()),
    ('Linear SVC', LinearSVC(max_iter=100))
]

# Метрики
metrics = {
    'accuracy': make_scorer(accuracy_score),
    'f1_score': make_scorer(f1_score)
}

# Список для хранения результатов
results1 = []

# Вычисление cross_val_score для каждого алгоритма и каждой метрики
for name, clf in classifiers:
    print(f"Classifier: {name}")
    for metric_name, metric in metrics.items():
        scores = cross_val_score(clf, X_transformed, y, cv=5, scoring=metric)
        mean_score = scores.mean()
        std_dev = scores.std() * 2
        print(f"{metric_name}: {mean_score:.4f} (+/- {std_dev:.4f})")
        # Добавление результатов в список
        results1.append((name, metric_name, mean_score, std_dev))
    print()

# Запись результатов в переменные
for result in results1:
    if result[0] == 'Logistic Regression':
        if result[1] == 'accuracy':
            acc_LR_del_missings = result[2]
        elif result[1] == 'f1_score':
            f1_LR_del_missings = result[2]
    elif result[0] == 'K-Nearest Neighbors':
        if result[1] == 'accuracy':
            acc_KNN_del_missings = result[2]
        elif result[1] == 'f1_score':
            f1_KNN_del_missings = result[2]
    elif result[0] == 'Linear SVC':
        if result[1] == 'accuracy':
            acc_SVM_del_missings = result[2]
        elif result[1] == 'f1_score':
            f1_SVM_del_missings = result[2]

# Вывод переменных с результатами
print(f"f1_LR: {f1_LR_del_missings}")
print(f"acc_LR: {acc_LR_del_missings}")
print(f"f1_KNN: {f1_KNN_del_missings}")
print(f"acc_KNN: {acc_KNN_del_missings}")
print(f"f1_SVM: {f1_SVM_del_missings}")
print(f"acc_SVM: {acc_SVM_del_missings}")

### Ваш код ###
# f1_LR_del_missings = 0
# acc_LR_del_missings = 0
# f1_KNN_del_missings = 0
# acc_KNN_del_missings = 0
# f1_SVM_del_missings = 0
# acc_SVM_del_missings = 0
results_classification.loc[7] = ['LogisticRegression_delete_missings', 'task15', f1_LR_del_missings, acc_LR_del_missings]
results_classification.loc[8] = ['KNeighborsClassifier_delete_missings', 'task15', f1_KNN_del_missings, acc_KNN_del_missings]
results_classification.loc[9] = ['LinearSVC_delete_missings', 'task15', f1_SVM_del_missings, acc_SVM_del_missings]
results_classification

Classifier: Logistic Regression


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

accuracy: 0.8507 (+/- 0.0057)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

f1_score: 0.9048 (+/- 0.0037)

Classifier: K-Nearest Neighbors
accuracy: 0.8249 (+/- 0.0030)
f1_score: 0.8872 (+/- 0.0019)

Classifier: Linear SVC


...

accuracy: 0.8511 (+/- 0.0038)


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


f1_score: 0.9053 (+/- 0.0028)

f1_LR: 0.90477272627845
acc_LR: 0.8506613731329669
f1_KNN: 0.8871526909906728
acc_KNN: 0.824945711538505
f1_SVM: 0.9052689910607421
acc_SVM: 0.85107085258329


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


,model,task,f1,accuracy
0,Most Frequent class,task10,0.864899,0.761958
1,LogisticRegression,task13,"[nan, 0.8978653530377668, 0.8955901426718547, ...","[nan, 0.8408393039918116, 0.8352098259979529, ..."
2,KNeighborsClassifier,task13,"[nan, 0.8794373744139318, 0.8829300196206671, ...","[nan, 0.8157625383828045, 0.8167860798362334, ..."
3,LinearSVC,task13,"[nan, 0.9031413612565444, 0.9032467532467534, ...","[nan, 0.8485158648925282, 0.8474923234390993, ..."
4,LogisticRegression_impute,task14,0.904642,0.850457
5,KNeighborsClassifier_impute,task14,0.887153,0.824946
6,LinearSVC_impute,task14,0.905142,0.850825
7,LogisticRegression_delete_missings,task15,0.904773,0.850661
8,KNeighborsClassifier_delete_missings,task15,0.887153,0.824946
9,LinearSVC_delete_missings,task15,0.905269,0.851071


In [ ]:
results_classification

,model,task,f1,accuracy
0,Most Frequent class,task10,0.864899,0.761958
1,LogisticRegression,task13,"[nan, 0.8978653530377668, 0.8955901426718547, ...","[nan, 0.8408393039918116, 0.8352098259979529, ..."
2,KNeighborsClassifier,task13,"[nan, 0.8794373744139318, 0.8829300196206671, ...","[nan, 0.8157625383828045, 0.8167860798362334, ..."
3,LinearSVC,task13,"[nan, 0.9031413612565444, 0.9032467532467534, ...","[nan, 0.8485158648925282, 0.8474923234390993, ..."
7,LogisticRegression_delete_missings,task15,0.901202,0.846911
8,KNeighborsClassifier_delete_missings,task15,0.882965,0.820574
9,LinearSVC_delete_missings,task15,0.902275,0.848326


 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [26]:


# Замена '?' на самые частые значения
for column in X.columns:
    if X[column].dtype == 'object':
        most_frequent = X[column].value_counts().index[0]
        X[column] = X[column].replace('?', most_frequent)

# Преобразование данных
X_transformed = pipeline.fit_transform(X)

# Алгоритмы классификации
classifiers = [
    ('Random Forest', RandomForestClassifier()),
    ('Gradient Boosting', GradientBoostingClassifier())
]

# Список для хранения результатов
results2 = []

# Вычисление cross_val_score для каждого алгоритма и каждой метрики
for name, clf in classifiers:
    print(f"Classifier: {name}")
    for metric_name, metric in metrics.items():
        scores = cross_val_score(clf, X_transformed, y, cv=5, scoring=metric)
        mean_score = scores.mean()
        std_dev = scores.std() * 2
        print(f"{metric_name}: {mean_score:.4f} (+/- {std_dev:.4f})")
        # Добавление результатов в список
        results2.append((name, metric_name, mean_score, std_dev))
    print()

# Запись результатов в переменные
for result in results2:
    if result[0] == 'Random Forest':
        if result[1] == 'accuracy':
            acc_RF = result[2]
        elif result[1] == 'f1_score':
            f1_RF = result[2]
    elif result[0] == 'Gradient Boosting':
        if result[1] == 'accuracy':
            acc_GB = result[2]
        elif result[1] == 'f1_score':
            f1_GB = result[2]

# Вывод переменных с результатами
print(f"f1_RF: {f1_RF}")
print(f"acc_RF: {acc_RF}")
print(f"f1_GB: {f1_GB}")
print(f"acc_GB: {acc_GB}")

# Добавление результатов в таблицу results_classification
results_classification.loc[10] = ['RandomForestClassifier', 'task16', f1_RF, acc_RF]
results_classification.loc[11] = ['GradientBoostingClassifier', 'task16', f1_GB, acc_GB]

# Вывод таблицы с результатами
print(results_classification)

# ### Ваш код ###

# f1_RF = 0
# acc_RF = 0
# f1_GB = 0
# acc_GB = 0
# results_classification.loc[10] = ['RandomForestClassifier', 'task16', f1_RF, acc_RF]
# results_classification.loc[11] = ['GradientBoostingClassifier', 'task16', f1_GB, acc_GB]

Classifier: Random Forest
accuracy: 0.8517 (+/- 0.0039)
f1_score: 0.9042 (+/- 0.0029)

Classifier: Gradient Boosting
accuracy: 0.8664 (+/- 0.0041)
f1_score: 0.9154 (+/- 0.0027)

f1_RF: 0.9042351441623605
acc_RF: 0.8516645736832039
f1_GB: 0.9153954075870411
acc_GB: 0.8663855852334714
                                   model    task  \
0                    Most Frequent class  task10   
1                     LogisticRegression  task13   
2                   KNeighborsClassifier  task13   
3                              LinearSVC  task13   
4              LogisticRegression_impute  task14   
5            KNeighborsClassifier_impute  task14   
6                       LinearSVC_impute  task14   
7     LogisticRegression_delete_missings  task15   
8   KNeighborsClassifier_delete_missings  task15   
9              LinearSVC_delete_missings  task15   
10                RandomForestClassifier  task16   
11            GradientBoostingClassifier  task16   

                                       

17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [30]:


# Загрузка данных и удаление строк с '?'
# data = pd.read_csv('adult.csv')
# X = data.drop(columns=["class", "class-num"])
# y = data["class-num"]

# Замена '?' на NaN и удаление строк с NaN
X = X.replace('?', np.nan).dropna()
y = y[X.index]

# Определение числовых и категориальных колонок
numeric_columns = X.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = X.select_dtypes(include=['object']).columns

# Создание преобразователей для числовых и категориальных колонок
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Заполнение пропусков медианой
    ('scaler', MinMaxScaler())  # Масштабирование данных
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Заполнение пропусков константой
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Кодирование категориальных данных
])

# Создание колонного преобразователя
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# Создание пайплайна
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Преобразование данных
X_transformed = pipeline.fit_transform(X)

# Классификаторы и их параметры для поиска по сетке
classifiers = {
    'Logistic Regression': (LogisticRegression(max_iter=100), {
        'C': [0.1, 1, 10],
        'penalty': ['l1', 'l2']
    }),
    'K-Nearest Neighbors': (KNeighborsClassifier(), {
        'n_neighbors': [3, 5, 7],
        'weights': ['uniform', 'distance']
    }),
    'Linear SVC': (LinearSVC(max_iter=100), {
        'C': [0.1, 1, 10],
        'penalty': ['l1', 'l2']
    }),
    'Random Forest': (RandomForestClassifier(), {
        'n_estimators': [10, 50, 100],
        'max_depth': [2, 3, 5],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2],
        'bootstrap': [True, False]
    }),
    'Gradient Boosting': (GradientBoostingClassifier(), {
        'n_estimators': [10, 50, 100],
        'learning_rate': [0.05, 0.1, 0.2],
        'max_depth': [2, 3, 5],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    })
}
# Список для хранения результатов
results = []

# Выполнение поиска по сетке для каждого классификатора
for name, (clf, param_grid) in classifiers.items():
    print(f"Classifier: {name}")
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='f1', n_jobs=-1)
    grid_search.fit(X_transformed, y)

    # Вывод лучших параметров и соответствующего результата
    print(f"Лучшие параметры: {grid_search.best_params_}")
    print(f"Лучший результат F1: {grid_search.best_score_:.4f}")

    # Сохранение результатов
    results.append((name, 'f1_score', grid_search.best_score_, grid_search.best_params_))
    print()

# Добавление результатов в таблицу results_classification
for result in results:
    results_classification.loc[len(results_classification)] = [result[0], 'task17', result[2], result[3]]

# Вывод таблицы с результатами
print(results_classification)


Classifier: Logistic Regression


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solve

Лучшие параметры: {'C': 10, 'penalty': 'l2'}
Лучший результат F1: 0.9050

Classifier: K-Nearest Neighbors
Лучшие параметры: {'n_neighbors': 7, 'weights': 'uniform'}
Лучший результат F1: 0.8908

Classifier: Linear SVC


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py", line 274, in fit
    self.coef_, self.intercept_, n_iter_ = _fit_liblinear(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py", line 1223, in _fit_liblinear
    solver_type = _get_liblinear_solver_type(multi_class, penalty,

Лучшие параметры: {'C': 0.1, 'penalty': 'l2'}
Лучший результат F1: 0.9052

Classifier: Random Forest
Лучшие параметры: {'bootstrap': True, 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Лучший результат F1: 0.9018

Classifier: Gradient Boosting
Лучшие параметры: {'learning_rate': 0.2, 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
Лучший результат F1: 0.9195

                                   model    task  \
0                    Most Frequent class  task10   
1                     LogisticRegression  task13   
2                   KNeighborsClassifier  task13   
3                              LinearSVC  task13   
4              LogisticRegression_impute  task14   
5            KNeighborsClassifier_impute  task14   
6                       LinearSVC_impute  task14   
7     LogisticRegression_delete_missings  task15   
8   KNeighborsClassifier_delete_missings  task15   
9              LinearSVC_delete_missings  ta

In [31]:
results_classification

,model,task,f1,accuracy
0,Most Frequent class,task10,0.864899,0.761958
1,LogisticRegression,task13,"[nan, 0.8978653530377668, 0.8955901426718547, ...","[nan, 0.8408393039918116, 0.8352098259979529, ..."
2,KNeighborsClassifier,task13,"[nan, 0.8794373744139318, 0.8829300196206671, ...","[nan, 0.8157625383828045, 0.8167860798362334, ..."
3,LinearSVC,task13,"[nan, 0.9031413612565444, 0.9032467532467534, ...","[nan, 0.8485158648925282, 0.8474923234390993, ..."
4,LogisticRegression_impute,task14,0.904642,0.850457
5,KNeighborsClassifier_impute,task14,0.887153,0.824946
6,LinearSVC_impute,task14,0.905142,0.850825
7,LogisticRegression_delete_missings,task15,0.904773,0.850661
8,KNeighborsClassifier_delete_missings,task15,0.887153,0.824946
9,LinearSVC_delete_missings,task15,0.905269,0.851071
